In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
dfn = pd.DataFrame()

In [3]:
for i in range(40):
    url = "https://www.metacritic.com/browse/games/score/metascore/all/pc/filtered?sort=desc&page="+str(i)+""
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',"Accept-Encoding": "*","Connection": "keep-alive"}
    try1 = requests.get(url,headers=header)
    s = BeautifulSoup(try1.content,'html.parser')
    
    all_titles = s.findAll('a',attrs={'class':'title'})
    titles = [title.get_text() for title in all_titles]
    ls_titles = []
    for title in titles:
        ls_titles.append(title.strip())
    
    all_metascores = s.findAll('div',attrs={'class':['metascore_w large game positive','metascore_w large game mixed','metascore_w large game negative','metascore_w large game tbd']})
    metascores = [metascore.get_text() for metascore in all_metascores]
    ls_metascores = []
    for metascore in metascores:
        ls_metascores.append(metascore.strip())
    ls_metascores = ls_metascores[::2]

    all_userscores = s.findAll('div',attrs={'class':['metascore_w user large game positive','metascore_w user large game negative','metascore_w user large game mixed','metascore_w user large game tbd']})
    userscores = [userscore.get_text() for userscore in all_userscores]
    ls_userscores = []
    for userscore in userscores:
        ls_userscores.append(userscore.strip())
    
    test = s.findAll('div',attrs={'class':'clamp-details'})
    ls_release = []
    for i in range(len(test)):
        ls_release.append(test[i].findAll('span')[2].get_text())
    
    all_summ = s.findAll('div',attrs={'class':'summary'})
    summaries = [sum.get_text() for sum in all_summ]
    ls_summary = []
    for summary in summaries:
        ls_summary.append(summary.strip())
    
    games = pd.DataFrame(list(zip(ls_titles,ls_summary,ls_release,ls_metascores,ls_userscores)))
    dfn = dfn.append(games)

In [8]:
dfn.head()

,0,1,2,3,4
0,Disco Elysium: The Final Cut,Disco Elysium - The Final Cut is the definitiv...,"March 30, 2021",97,8.2
1,Half-Life 2,[Metacritic's 2004 PC Game of the Year] By ta...,"November 16, 2004",96,9.2
2,Grand Theft Auto V,Los Santos: a sprawling sun-soaked metropolis ...,"April 13, 2015",96,7.8
3,The Orange Box,Games included in The Orange Box compilation: ...,"October 10, 2007",96,9.3
4,Half-Life,Half-Life combines great storytelling in the t...,"November 19, 1998",96,9.1


In [5]:
dfn.shape

(4000, 5)

In [6]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 99
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4000 non-null   object
 1   1       4000 non-null   object
 2   2       4000 non-null   object
 3   3       4000 non-null   object
 4   4       4000 non-null   object
dtypes: object(5)
memory usage: 187.5+ KB


In [9]:
dfn.isnull().sum()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [10]:
dfn.rename(columns={0:'Title',1:'Description',2:'Release Date',3:'Metascore',4:'User Score'},inplace=True)

In [12]:
dfn.head()

,Title,Description,Release Date,Metascore,User Score
0,Disco Elysium: The Final Cut,Disco Elysium - The Final Cut is the definitiv...,"March 30, 2021",97,8.2
1,Half-Life 2,[Metacritic's 2004 PC Game of the Year] By ta...,"November 16, 2004",96,9.2
2,Grand Theft Auto V,Los Santos: a sprawling sun-soaked metropolis ...,"April 13, 2015",96,7.8
3,The Orange Box,Games included in The Orange Box compilation: ...,"October 10, 2007",96,9.3
4,Half-Life,Half-Life combines great storytelling in the t...,"November 19, 1998",96,9.1


In [13]:
dfn.to_csv('PC_Games_2021.csv')